In [46]:
# 反向传播的目的：优化模型参数
# 损失函数（loss）：预测值（y）与已知答案（y_）的差距
#         均方误差（MSE）loss = tf.reduce_mean(tf.square(y_-y))
# 反向传播训练方法：以减小loss值为优化目标
#         tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)梯度下降
#             　　　　标准梯度下降法(GD, Gradient Descent)
#                         基本策略可以理解为”在有限视距内寻找最快路径下山“，因此每走一步，参考当前位置最陡的方向(即梯度)进而迈出下一步
#                         评价：标准梯度下降法主要有两个缺点:
#                                 训练速度慢：
#                                 每走一步都要要计算调整下一步的方向，下山的速度变慢
# 　　　　　　　　                在应用于大型数据集中，每输入一个样本都要更新一次参数，且每次迭代都要遍历所有的样本
# 　　　　　　　　                会使得训练过程及其缓慢，需要花费很长时间才能得到收敛解
#                               容易陷入局部最优解：
# 　　　　　　　　                由于是在有限视距内寻找下山的反向
# 　　　　　　　　                当陷入平坦的洼地，会误以为到达了山地的最低点，从而不会继续往下走
# 　　　　　　　　                所谓的局部最优解就是鞍点。落入鞍点，梯度为0，使得模型参数不在继续更新
                        
#                     随机梯度下降法(SGD, Stochastic Gradient Descent)
#                         基本策略可以理解为，在下山之前掌握了附近的地势情况，选择总体平均梯度最小的方向下山
#                         评价：
#                                 批量梯度下降法比标准梯度下降法训练时间短，且每次下降的方向都很正确
                            
#                     批量梯度下降法(BGD, Batch Gradient Descent)
#                         基本策略可以理解为随机梯度下降像是一个盲人下山，不用每走一步计算一次梯度，但是他总能下到山底，只不过过程会显得扭扭曲曲
#                         评价：
#                           优点：
#                                 虽然SGD需要走很多步的样子，但是对梯度的要求很低（计算梯度快）。而对于引入噪声，大量的理论和实践工作证明，只要噪声不是特别大，SGD都能很好地收敛。
#                                 应用大型数据集时，训练速度很快。比如每次从百万数据样本中，取几百个数据点，算一个SGD梯度，更新一下模型参数。
#                                 相比于标准梯度下降法的遍历全部样本，每输入一个样本更新一次参数，要快得多。
#                           缺点：
#                                 SGD在随机选择梯度的同时会引入噪声，使得权值更新的方向不一定正确。
#                                 此外，SGD也没能单独克服局部最优解的问题。
#                                 由于这种方法是在一次更新中，就对整个数据集计算梯度，
#                                 所以计算起来非常慢，遇到很大量的数据集也会非常棘手，而且不能投入新数据实时更新模型
                                
#         tf.train.MomentumOptimizer(learning_rate).minimize(loss)
#                 标准动量优化方法Momentum
#                 理解策略为：由于当前权值的改变会受到上一次权值改变的影响，类似于小球向下滚动的时候带上了惯性。
#                             这样可以加快小球向下滚动的速度。
                
#                 牛顿加速梯度（NAG, Nesterov accelerated gradient）算法
#                 理解策略：在Momentun中小球会盲目地跟从下坡的梯度，容易发生错误。
#                           所以需要一个更聪明的小球，能提前知道它要去哪里，还要知道走到坡底的时候速度慢下来而不是又冲上另一个坡。
                    
#         tf.train.AdamOptimizer(learning_rate).minimize(loss)
#                 自适应矩估计（Adaptive Moment Estimation）
#                 概率论中矩的含义是：如果一个随机变量 X 服从某个分布，X 的一阶矩是 E(X)，也就是样本平均值，X 的二阶矩就是 E(X^2)，
#                                     也就是样本平方的平均值。
#                 Adam 算法利用梯度的一阶矩估计和二阶矩估计动态调整每个参数的学习率。
#                 评价：
#                         相比于SGD算法：
#                                         不容易陷于局部优点
#                                         速度更快，学习效果更为有效
#                                         纠正其他优化技术中存在的问题，如学习率消失或是高方差的参数更新导致损失函数波动较大等问题。
#                                         Adam 的调参相对简单，默认参数就可以处理绝大部分的问题。
                                    
#         学习率：每次参数更新的幅度
    
#导入模块，生成模拟数据集
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
BATCH_SIZE = 4   #一次喂入神经网络8组数据
seed = 23455     #随机种子

In [47]:
#基于seed产生随机数
rng = np.random.RandomState(seed)

In [48]:
#随机数返回32行2列的矩阵 表示32组 体积和重量 作为输入数据集
X = rng.rand(32,2)

In [49]:
Y = [[int(x0 + x1<1)] for (x0,x1) in X]
print("X:\n",X)
print("Y:\n",Y)

X:
 [[0.83494319 0.11482951]
 [0.66899751 0.46594987]
 [0.60181666 0.58838408]
 [0.31836656 0.20502072]
 [0.87043944 0.02679395]
 [0.41539811 0.43938369]
 [0.68635684 0.24833404]
 [0.97315228 0.68541849]
 [0.03081617 0.89479913]
 [0.24665715 0.28584862]
 [0.31375667 0.47718349]
 [0.56689254 0.77079148]
 [0.7321604  0.35828963]
 [0.15724842 0.94294584]
 [0.34933722 0.84634483]
 [0.50304053 0.81299619]
 [0.23869886 0.9895604 ]
 [0.4636501  0.32531094]
 [0.36510487 0.97365522]
 [0.73350238 0.83833013]
 [0.61810158 0.12580353]
 [0.59274817 0.18779828]
 [0.87150299 0.34679501]
 [0.25883219 0.50002932]
 [0.75690948 0.83429824]
 [0.29316649 0.05646578]
 [0.10409134 0.88235166]
 [0.06727785 0.57784761]
 [0.38492705 0.48384792]
 [0.69234428 0.19687348]
 [0.42783492 0.73416985]
 [0.09696069 0.04883936]]
Y:
 [[1], [0], [0], [1], [1], [1], [1], [0], [1], [1], [1], [0], [0], [0], [0], [0], [0], [1], [0], [0], [1], [1], [0], [1], [0], [1], [1], [1], [1], [1], [0], [1]]


In [50]:
#定义NN输入、参数、输出
x = tf.placeholder(tf.float32,shape=(None,2))
y_ = tf.placeholder(tf.float32,shape=(None,1))
w1 = tf.Variable(tf.random.normal([2,3],stddev=1,seed=1))
w2 = tf.Variable(tf.random.normal([3,1],stddev=1,seed=1))
a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

In [51]:
#定义损失函数及反向传播方法
loss = tf.reduce_mean(tf.square(y-y_))
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(loss)
#train_step = tf.train.MomentumOptimizer(0.001).minimize(loss)
#train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

In [52]:
#生成会话，训练steps轮
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print("##########未经训练w1、w2############")
    print("w1:\n",sess.run(w1))
    print("w2:\n",sess.run(w2))
    
    #训练模型
    STEPS = 10000
    for i in range(STEPS):
        start = (i*BATCH_SIZE) % 32
        end = start + BATCH_SIZE
        sess.run(train_step,feed_dict={x:X[start:end],y_: Y[start:end]})
        if i % 500 == 0:
            total_loss = sess.run(loss,feed_dict={x: X,y_ : Y})
            print("After %d traning step(s),loss on all data is %g"%(i,total_loss))
        
    #输出训练后的参数值
    print("w1:\n",sess.run(w1))
    print("w2:\n",sess.run(w2))

##########未经训练w1、w2############
w1:
 [[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
w2:
 [[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]
After 0 traning step(s),loss on all data is 5.14196
After 500 traning step(s),loss on all data is 0.428969
After 1000 traning step(s),loss on all data is 0.409785
After 1500 traning step(s),loss on all data is 0.399936
After 2000 traning step(s),loss on all data is 0.394156
After 2500 traning step(s),loss on all data is 0.390617
After 3000 traning step(s),loss on all data is 0.388349
After 3500 traning step(s),loss on all data is 0.386875
After 4000 traning step(s),loss on all data is 0.385874
After 4500 traning step(s),loss on all data is 0.385205
After 5000 traning step(s),loss on all data is 0.384728
After 5500 traning step(s),loss on all data is 0.384408
After 6000 traning step(s),loss on all data is 0.384167
After 6500 traning step(s),loss on all data is 0.38401
After 7000 traning step(s),loss on all data is 0.383883